In [1]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Replace 'your_path' with the path where you saved Gesture.zip in Google Drive, or upload directly to Colab.
# Example path for files on Google Drive: '/content/drive/MyDrive/path_to_files/gesture/train.pt'
train_path = '/content/drive/MyDrive/Gesture/train.pt'
val_path = '/content/drive/MyDrive/Gesture/val.pt'
test_path = '/content/drive/MyDrive/Gesture/test.pt'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch scikit-learn


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt


In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load the .pt files
train_data = torch.load(train_path)
val_data = torch.load(val_path)
test_data = torch.load(test_path)

# Extract features (samples) and labels
X_train = train_data['samples']
y_train = train_data['labels']
X_val = val_data['samples']
y_val = val_data['labels']
X_test = test_data['samples']
y_test = test_data['labels']

# Convert data to float32 and labels to long for PyTorch compatibility
X_train = X_train.float()
y_train = y_train.long()
X_val = X_val.float()
y_val = y_val.long()
X_test = X_test.float()
y_test = y_test.long()

# Impute missing values with the mean for each feature
imputer = SimpleImputer(strategy='mean')
X_train_imputed = torch.tensor(imputer.fit_transform(X_train.view(X_train.shape[0], -1).numpy()), dtype=torch.float32).view(X_train.shape)
X_val_imputed = torch.tensor(imputer.transform(X_val.view(X_val.shape[0], -1).numpy()), dtype=torch.float32).view(X_val.shape)
X_test_imputed = torch.tensor(imputer.transform(X_test.view(X_test.shape[0], -1).numpy()), dtype=torch.float32).view(X_test.shape)

# Scale the data
scaler = StandardScaler()
X_train_scaled = torch.tensor(scaler.fit_transform(X_train_imputed.view(X_train_imputed.shape[0], -1).numpy()), dtype=torch.float32).view(X_train_imputed.shape)
X_val_scaled = torch.tensor(scaler.transform(X_val_imputed.view(X_val_imputed.shape[0], -1).numpy()), dtype=torch.float32).view(X_val_imputed.shape)
X_test_scaled = torch.tensor(scaler.transform(X_test_imputed.view(X_test_imputed.shape[0], -1).numpy()), dtype=torch.float32).view(X_test_imputed.shape)

# Create DataLoaders
train_dataset = TensorDataset(X_train_scaled, y_train)
val_dataset = TensorDataset(X_val_scaled, y_val)
test_dataset = TensorDataset(X_test_scaled, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Optional: Check DataLoader outputs
for data, label in train_loader:
    print("Data batch shape:", data.shape)
    print("Label batch shape:", label.shape)
    break


<ipython-input-4-b80679d3d9da>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(train_path)
<ipython-input-4-b80679d3d9da>:8: FutureWarning: You are 

Data batch shape: torch.Size([32, 3, 206])
Label batch shape: torch.Size([32])


In [5]:
import torch
import torch.nn as nn

class SelfSupervisedModel(nn.Module):
    def __init__(self):
        super(SelfSupervisedModel, self).__init__()
        # Update the in_channels of the first conv layer to 3
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 206, 128)  # Flattened size adjustment
        self.fc2 = nn.Linear(128, 64)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = x.view(x.size(0), -1)  # Flatten before passing to fully connected layers
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# Initialize model, criterion, and optimizer
model = SelfSupervisedModel()
criterion = nn.CosineEmbeddingLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)  # Added scheduler

# Self-supervised pre-training (extended to 20 epochs)
pretrain_epochs = 5
for epoch in range(pretrain_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, _) in enumerate(train_loader):
        # Separate positive and negative pairs
        pos_pair = data
        neg_pair = data[torch.randperm(data.size(0))]  # Random shuffling to create negatives

        # Ensure target size matches batch size
        target = torch.ones(pos_pair.size(0), device=data.device)  # Positive labels for CosineEmbeddingLoss

        optimizer.zero_grad()

        # Forward pass
        output_pos = model(pos_pair)
        output_neg = model(neg_pair)

        # Compute CosineEmbeddingLoss
        loss = criterion(output_pos, output_neg, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Step the learning rate scheduler
    scheduler.step()

    print(f'Epoch [{epoch+1}/{pretrain_epochs}], Loss: {running_loss/len(train_loader):.4f}')


Epoch [1/5], Loss: 0.0171
Epoch [2/5], Loss: 0.0006
Epoch [3/5], Loss: 0.0003
Epoch [4/5], Loss: 0.0002
Epoch [5/5], Loss: 0.0001


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define your model class with dynamic calculation of the flattened dimension
class YourModel(nn.Module):
    def __init__(self):
        super(YourModel, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(p=0.5)  # Dropout for regularization

        # Calculate the flattened dimension
        sample_input = torch.randn(1, 3, 206)  # Sample input with your data's dimensions
        flattened_dim = self._get_flattened_dim(sample_input)

        # Define the fully connected layer with correct input size
        self.fc = nn.Linear(flattened_dim, 10)  # Assuming 10 output classes

    def _get_flattened_dim(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        return x.size(1)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)  # Flatten for fully connected layer
        x = self.fc(x)
        return x

# Instantiate and check the model
model = YourModel()
print(model)


YourModel(
  (conv1): Conv1d(3, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=13184, out_features=10, bias=True)
)


In [16]:
# Loss and Optimizer with Weight Decay for regularization
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# DataLoader for training, validation, and test sets (modify as per your dataset)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Evaluation Function for Validation and Test
def evaluate(model, loader):
    model.eval()  # Set model to evaluation mode
    loss, correct, total = 0, 0, 0
    all_labels, all_preds = [], []

    with torch.no_grad():
        for data, labels in loader:
            outputs = model(data)
            batch_loss = criterion(outputs, labels)
            loss += batch_loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    accuracy = correct / total
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return loss / len(loader), accuracy, precision, recall, f1

# Training Loop with Early Stopping
num_epochs = 50
best_val_loss = float('inf')
patience, patience_counter = 5, 0

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    train_loss, correct, total = 0, 0, 0

    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = correct / total
    val_loss, val_accuracy, _, _, _ = evaluate(model, val_loader)  # Only val_loss and val_accuracy are printed

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

    # Early Stopping Check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model = model.state_dict()  # Save best model
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

model.load_state_dict(best_model)  # Load best model for testing

# Final Testing
test_loss, test_accuracy, test_precision, test_recall, test_f1 = evaluate(model, test_loader)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test Precision: {test_precision:.4f}, Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}")


Epoch [1/50], Train Loss: 0.6530, Train Acc: 0.9812, Val Loss: 3.5397, Val Acc: 0.5917
Epoch [2/50], Train Loss: 0.7269, Train Acc: 0.9781, Val Loss: 3.8861, Val Acc: 0.5833
Epoch [3/50], Train Loss: 0.5001, Train Acc: 0.9875, Val Loss: 3.5757, Val Acc: 0.5417
Epoch [4/50], Train Loss: 0.3913, Train Acc: 0.9875, Val Loss: 3.6619, Val Acc: 0.5583
Epoch [5/50], Train Loss: 0.5884, Train Acc: 0.9781, Val Loss: 3.6401, Val Acc: 0.5417
Epoch [6/50], Train Loss: 0.3672, Train Acc: 0.9875, Val Loss: 3.8715, Val Acc: 0.5833
Early stopping triggered.
Test Loss: 3.8715, Test Accuracy: 0.5833, Test Precision: 0.5366, Test Recall: 0.5833, Test F1: 0.5524
